### Projeto 1 - IBM - RH - Análise de desempenho e atritos de funcionários

In [45]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


In [8]:
dados = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
dados['turnover'] = dados['Attrition'].replace({'Yes': 1, 'No': 0})


In [62]:
# fase 1: convertendo string para inteiro
# referência do código: João Rodrigues

# convertendo BusinessTravel para inteiro
dados['BusinessTravel'] = dados['BusinessTravel'].replace(
    {'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})

# convertendo
# EDUCATION FIELD
# 1=HR, 2=LIFE SCIENCES, 3=MARKETING, 4=MEDICAL SCIENCES, 5=OTHERS, 6= TEHCNICAL
dados['EducationField'] = dados['EducationField'].replace(
    {'Human Resources': 1, 'Life Sciences': 2, 'Marketing': 3, 'Medical': 4, 'Other': 5, 'Technical Degree': 6})

# GENDER
# 1=FEMALE, 2=MALE
dados['Gender'] = dados['Gender'].replace({'Female': 1, 'Male': 2})

# department
#     (1=HR, 2=R&D, 3=Sales)
dados['Department'] = dados['Department'].replace(
    {'Sales': 3, 'Research & Development': 2, 'Human Resources': 1})

# job role
#    (1=HC REP, 2=HR, 3=LAB TECHNICIAN, 4=MANAGER, 5= Manufacturing Director, 6= REASEARCH DIRECTOR, 7= RESEARCH SCIENTIST, 8=SALES EXECUTIEVE, 9= SALES REPRESENTATIVE)
dados['JobRole'] = dados['JobRole'].replace({'Sales Executive': 8, 'Research Scientist': 6, 'Laboratory Technician': 3,
                                             'Manufacturing Director': 5, 'Healthcare Representative': 1, 'Manager': 4,
                                             'Sales Representative': 9, 'Research Director': 6, 'Human Resources': 2})

# marital status
# (1=DIVORCED, 2=MARRIED, 3=SINGLE
dados['MaritalStatus'] = dados['MaritalStatus'].replace(
    {'Married': 2, 'Single': 3, 'Divorced': 1})

# over 18
# 1=YES, 2=NO)
dados['Over18'] = dados['Over18'].replace({'Y': 1, 'N': 2})

# overtime
#     (1=NO, 2=YES)
dados['OverTime'] = dados['OverTime'].replace({'Yes': 2, 'No': 1})

X = dados.drop(['Attrition', 'turnover'], axis=1)
y = dados['turnover']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)


----

## Exercicio 1. (2 pontos)
Realize uma análise sobre as variáveis independentes em relação a variável turnover. Quais ideias você obteve? Faça uma pequena apresentação de três a quatro gráficos e escreva suas percepções abaixo de cada um.

----

## Exercicio 2. (1 ponto)
Faça seleção de características e selecione as 8 variáveis mais importantes. Escreva quais são elas e destaque as variáveis que estão presentes nas ideias apresentadas no exercício 1

In [24]:
selector = RFE(LogisticRegression(random_state=42, max_iter=2000),
               n_features_to_select=8, step=1, verbose=0, importance_getter='auto')
selector = selector.fit(X_train, y_train)


In [25]:
selector.get_feature_names_out()


array(['BusinessTravel', 'Department', 'Gender', 'JobInvolvement',
       'JobLevel', 'JobSatisfaction', 'MaritalStatus', 'OverTime'],
      dtype=object)

----

## Exercicio 3. (2 pontos)
Construa um modelo de árvore de decisão. Obtenha os valores das métricas: acurácia, precisão e sensitividade.

In [59]:
dt = DecisionTreeClassifier(random_state=10, max_depth=9)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)


In [61]:
# agora uma função que vai printar bonitinho as métricas
# referência do código: João Rodrigues
def print_metrics(model_name: str, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the accuracy, precision and recall scores of the model

      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA

      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.

  """
  return (f''' 
    Acurácia do Modelo {model_name} sem PCA: {accuracy_score(y_test, y_pred):.4f}
    

    Precisão do Modelo {model_name} sem PCA: {precision_score(y_test, y_pred, average='macro'):.4f}
    

    Recall (Sensibilidade) do Modelo {model_name} sem PCA: {recall_score(y_test, y_pred, average='macro'):.4f}
   ''')

In [54]:
print(print_metrics(dt ,y_test, y_pred))

 
    Acurácia do Modelo DecisionTreeClassifier(max_depth=11, random_state=10) sem PCA: 0.7959
    

    Precisão do Modelo DecisionTreeClassifier(max_depth=11, random_state=10) sem PCA: 0.5977
    

    Recall (Sensibilidade) do Modelo DecisionTreeClassifier(max_depth=11, random_state=10) sem PCA: 0.6132
   


----

## Exercicio 4. (1 ponto)
Faça a representação da árvore e observe como foi realizado cada ramificação. Explique o que é GINI e quais seriam as desvantagens?

In [60]:
plt.figure(figsize=[200, 100])
plot_tree(dt)
plt.show()

## Explique o que é GINI e quais seriam as desvantagens?

O funcionamento interno da impureza Gini também é um tanto semelhante ao funcionamento da entropia na Árvore de Decisão. No algoritmo da árvore de decisão, ambos são usados ​​para construir a árvore, dividindo de acordo com os recursos apropriados, mas há uma grande diferença no cálculo de ambos os métodos.  


<img src = 'Gini-Impurity-vs-Entropy.png' >   
 
O funcionamento interno de ambos os métodos é muito semelhante e ambos são usados ​​para calcular o recurso / divisão após cada nova divisão. Mas se compararmos os dois métodos, a Impureza de Gini é mais eficiente do que a entropia em termos de poder de computação. Como você pode ver no gráfico para entropia, ela primeiro aumenta até 1 e depois começa a diminuir, mas no caso da impureza de Gini ela só vai até 0,5 e depois começa a diminuir, portanto, requer menos poder computacional. O intervalo de Entropia está entre 0 e 1 e o intervalo de Impureza Gini está entre 0 e 0,5. Portanto, podemos concluir que a Impureza de Gini é melhor do que a entropia para selecionar as melhores características

----

## Exercicio 5. (3 pontos)
Chamamos de overfitting (superajuste) a situação onde um modelo "decora" como predizer o conjunto de treino, mas tende a errar muito ao predizer o conjunto de teste. Dizemos que este modelo não generaliza, não consegue lidar com dados novos. Vimos dois modelos de diferentes estrategias de aprendizado por agrupamento durante as aulas. Qual destes modelos tem maior chance de sofrer um overfitting e por quê?


Resposta :   
As árvores de decisão possuem inúmeras vantagens, tais como a flexibilidade em ser usado em conjuntos de dados contínuos ou categóricos ou composto, para situações de classificação ou regressão. É um modelo robusto e facilmente interpretável.   
Os problemas com o uso de árvore de decisão são bem conhecidos e documentados. O problema de valor ausente, é quando um valor desconhecido é submetido a predição da árvore de decisão e causa o problema em decidir qual ramo a árvore de decisão deverá prosseguir. Há também um gargalo na árvore de decisão quando há muitas variáveis de valor contínuo, onde grande parte do tempo de processamento da árvore de decisão será em ordenar os valores de cada variável contínua. Outro ponto conhecido é o fator de instabilidade, devido a pequenas variações no conjunto de dados podem acarretar grandes variações na árvore final.

----

## Exercicio 6. (1 ponto)
Você recomendaria o seu modelo para uma empresa utilizar em seu setor de Recursos Humanos? Como você explicaria para um setor que não possui conhecimento algum sobre Ciência de Dados que um modelo com 100% de acurácia é impossível de se obter?

Resposta :  
Vamos lá professor, eu não recomendaria nunca o meu modelo, mas vamos lá   

Ter um alto valor de acurácia é bom, mas quando estiver mais proximo de 100% o risco do modelo não saber trabalhar com novos dados é alta, a explicação é ele ficou muito justo aos dados de treinamento e não sabe lidar com os novos tornado o modelo inutilizavel;